In [1]:
import pandas as pd
import numpy as np
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
import datetime
import openai
import os
import json

In [79]:
llm_model = "gpt-4"
api_key = 'put your api key here'
data_date = "04-13"
path = "../Users/"
openai.api_key = api_key

In [6]:
def read_fixed_covariates(date):
    df1 = pd.read_csv(f'{path}users_data_one_2023-{date}.csv', engine='python')
    df2 = pd.read_csv(f'{path}users_data_two_2023-{date}.csv', engine='python')
    df2.drop([228, 229], inplace=True)
    df = pd.concat([df1, df2], ignore_index=True)
    df.reset_index(drop=True, inplace=True)
    df["id"] = df["id"].astype(str)
    # print(df.shape)
    return df
data = read_fixed_covariates(data_date)

In [56]:
def extract_location(lst):
    prompt=f"""
    Your task is to extract and complete the location for a list of sentences.
    If location found return the country and city for this location in the format:
    For example, from sentence "We visited Paris last summer and it was wonderful." the results is {{"country": "France", "city": "Paris"}}.
    Ff no location found then return a dictionary {{"country": null, "city": null}}
    The output should be a list of dictionaries in the format: (only return the list)
    {{[{{"country": "France", "city": "Paris"}}, {{"country": null, "city": null}}, ...]}}
    sentence: ```{lst}```
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=llm_model,
        messages=messages,
        temperature=0,
    )
    
    try:
        # Extracting and returning the first choice.
        location = response.choices[0].message["content"]
        print(f"Extracted Location: {location}")
        return location
    except IndexError:
        # Handle error/No location found
        return None


In [58]:
whole_df = read_fixed_covariates(data_date)
non_na = whole_df[whole_df["location"].notna()]
non_na.reset_index(drop=True, inplace=True)
non_na.head()
non_na.shape

(1254, 19)

In [71]:
llm = OpenAI(temperature=0, openai_api_key=api_key)

In [81]:
# Process by batch of 50
import time
batch_size = 50
num_batches = int(np.ceil(non_na.shape[0] / batch_size))
results = []
for i in range(num_batches):
    print(f"Processing batch {i+1} of {num_batches}")
    start = i * batch_size
    end = min((i + 1) * batch_size, non_na.shape[0])
    print(f"Start: {start}, End: {end}")
    current_batch = non_na["location"].tolist()[start:end]
    # llm.get_num_tokens(current_batch)
    location = extract_location(current_batch)
    location = location.replace("None", "null")
    location_lst = json.loads(location)
    results.extend(location_lst)
    print(f"Results: {location_lst}")
    time.sleep(10)
    


Processing batch 1 of 26
Start: 0, End: 50
Extracted Location: [{"country": "South Korea", "city": "Seoul"}, {"country": "Canada", "city": "Toronto"}, {"country": "Sierra Leone", "city": null}, {"country": null, "city": null}, {"country": "USA", "city": "Brooklyn"}, {"country": null, "city": null}, {"country": "USA", "city": "South Florida"}, {"country": "Australia", "city": "Brisbane"}, {"country": "USA", "city": null}, {"country": "England", "city": "London"}, {"country": null, "city": null}, {"country": null, "city": null}, {"country": null, "city": null}, {"country": "United Kingdom", "city": "Hereford"}, {"country": null, "city": null}, {"country": "USA", "city": "Montgomery"}, {"country": "USA", "city": "California"}, {"country": "USA", "city": null}, {"country": "USA", "city": "Washington"}, {"country": "Malaysia", "city": null}, {"country": "USA", "city": "Everett"}, {"country": "USA", "city": "Fairfax"}, {"country": "USA", "city": "Suffield"}, {"country": "USA", "city": "NYC"}

In [169]:
non_na["gpt_country"] = [x["country"] for x in results]
non_na["gpt_city"] = [x["city"] for x in results]
final_df = whole_df.merge(non_na[["id", "gpt_country", "gpt_city"]], on="id", how="left")

In [6]:
data = pd.read_csv("finallocation_df.csv")

In [8]:
to_json = data[['id', 'gpt_country', 'gpt_city']]

In [10]:
to_json.to_json("location.json", orient="records")